In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
!pip install rdflib

In [2]:
from Extractors import *
from rdflib import Graph, RDF, URIRef
from rdflib.namespace import RDF, XSD, FOAF, Namespace

In [3]:
graphe = Graph()
graphe.parse(location="TTLFiles/field.ttl", format="turtle")

<Graph identifier=N88e3f789bd2144e68ca351a3c20a1448 (<class 'rdflib.graph.Graph'>)>

In [4]:
graphe.parse(location="TTLFiles/author.ttl", format="turtle")
graphe.parse(location="TTLFiles/paper.ttl", format="turtle")
graphe.parse(location="TTLFiles/acemap.ttl", format="turtle")
graphe.parse(location="TTLFiles/affiliation.ttl", format="turtle")
graphe.parse(location="TTLFiles/conference.ttl", format="turtle")
graphe.parse(location="TTLFiles/institute.ttl", format="turtle")
graphe.parse(location="TTLFiles/journal.ttl", format="turtle")

<Graph identifier=N88e3f789bd2144e68ca351a3c20a1448 (<class 'rdflib.graph.Graph'>)>

In [5]:
ace = Namespace('http://www.semanticweb.org/acemap#')

In [6]:
def AjouterPrefixe(prefixe, chaine) :
    """
    Fonction qui ajouter un préfixe à une chaine de caractère et remplace les " " par des "_"
    retourne une chaine de caractères avec le préfixe placé devant
    exemple : AjouterPrefixe("AUTH", "Albert Einstein") retourne "AUTH_Albert_Einstein"
    """
    pref = prefixe + "_"
    new_chaine = pref + chaine.replace(" ", "_")
    return new_chaine

def YearFromDate(date) :
    """
    Extrait l'année d'un date qui est au format xsd.date (YYYY-MM-DD)
    Retourne la chaine de caractère correspondant à l'année de la date
    """
    return date[:4]  


def ExtraireAuteurs(graphe) :
    """
    Extrait tous les id d'auteurs dans un tableau
    prends un paramètre un graphe de rdflib
    retourne un tableau avec tous les id des auteurs
    """
    auteurs = []
    authors = graphe.subjects(RDF.type, ace.Author)
    for author in authors :
        auteurs.append(author)
    return auteurs


def ExtrairePublication(graphe) :
    """
    Extrait tous les id de publication dans un tableau
    prends un paramètre un graphe de rdflib
    retourne un tableau avec tous les id des publications
    """
    publi = []
    papers = graphe.subjects(RDF.type, ace.Paper)
    for paper in papers :
        publi.append(paper)
    return publi


def ExtraireNomAuteurs(graphe, auteurs) :
    """
    Extrait les identifiants des auteurs et leurs noms présents dans acemap
    prend en paramètre un objet graphe de la RDFlib
    retourne une dictionnaire qui associe authorID à son nom
    """

    IDToAuthors = dict()
    for auteur in auteurs :
        noms = graphe.objects(auteur, ace.author_name)
        for nom in noms :
            IDToAuthors[auteur] = nom
    return IDToAuthors

def ExtraireConceptes(graphe, auteurs):
    """
    Extrait les domaines des auteurs
    prend en paramètre un objet graphe de rdflib et un liste d'id d'autheurs sous forme d'URI
    retourne un dictionnaire associant à un authorID tous les FieldID auxquelles il est rataché
    { authorID : [fieldID, fieldID, fieldID], authorID : [fieldID, ...] ... }
    """
    authIDToFieldID = dict()
    for auteur in auteurs :
        domaines = graphe.objects(auteur, ace.Field)
        authIDToFieldID[auteur] = []
        for domaine in domaines :
            authIDToFieldID[auteur].append(domaine)
    return authIDToFieldID

def ExtrairesDate(graphe, auteurs):
    """
    Extrait les dates de publication d'un auteur.
    prend en paramètre un graphe rdf et une liste d'authorID sous forme d'URI
    retourne un dictionnaire associant des authorID à des dates de publication 
    { authorID : [date, date, date], authorID : [date, ...] ... }
    """
    authIDToYears = dict()
    for auteur in auteurs :
        authIDToYears[auteur] = []
        #on va générer la liste de toutes les publication qui ont été ecrite par l'auteur en question
        publications = graphe.subjects(ace.paper_is_written_by, auteur)
        for publication in publications :
            dates = graphe.objects(publication, ace.paper_publish_date)
            for date in dates :
                authIDToYears[auteur].append(date)
    return authIDToYears

def ExtraireAuteursPubli(graphe, publications) :
    """
    Extrait les auteurs pour toutes les publication
    prend en parametre un graphe rdf
    retourne un dictionnaire qui associe les paperID a tous ses authorID
    { paperID : [authorID, authoID, authorID], paperID : [authorID, ...] ... }
    """
    paperIDToAuthorID = dict()
    for publication in publications :
        auteurs = graphe.objects(publication, ace.paper_is_written_by)
        paperIDToAuthorID[publication] = []
        for auteur in auteurs :
            paperIDToAuthorID[publication].append(auteur)
    return paperIDToAuthorID


def Coauteurs(PaperToAuthor, auteurs, publications):
    """
    Crée  un dictionnaire associant chaque auteur à ses coauteurs
    prend en paramètre un dictionnaire associant les publication à leurs auteurs (cf ExtraireAuteursPubli), un tableau des auteurs et un tableau des publications
    retourne un dictionnaire sous la forme { authID : [authID, authID ...] , authID : ......}
    """
    coaut = dict()
    for auteur in auteurs :
        coaut[auteur] = []
        for publication in publications :
            if auteur in PaperToAuthor[publication] :
                for aut in PaperToAuthor[publication] :
                    if (aut != auteur) and (aut not in coaut[auteur]) :
                        coaut[auteur].append(aut)
    return coaut

In [12]:
auteurs = ExtraireAuteurs(graphe)
publications = ExtrairePublication(graphe)
IDtoAuthor = ExtraireNomAuteurs(graphe, auteurs)
authToField = ExtraireConceptes(graphe, auteurs)

In [13]:
for auteur in auteurs:
    print(auteur)
print("-----------")
for publication in publications :
    print(publication)
print("-----------")


http://www.semanticweb.org/acemap#791D05BF
http://www.semanticweb.org/acemap#218FC062
-----------
http://www.semanticweb.org/acemap#7BFA9BE5
http://www.semanticweb.org/acemap#7CE9EBAC


In [ ]:
autann = ExtraireAnnees(graphe, IDtoAuthor.keys())
for key in autann.keys() :
    for ann in autann[key] :
        print("---------")
        print(key)
        print(ann)
        print(type(ann))
        print(Annee(ann))
        print(type(Annee(ann)))
        
publiAut = ExtraireAuteursPubli(graphe)
for key in publiAut.keys() :
    for aut in publiAut[key]:
        print("-------")
        print(key)
        print(aut)
        print(type(aut))
        print(IDtoAuthor[aut])
    